In [124]:
import jsonlines
from collections import defaultdict
import openai
import json
import regex
from typing import Dict, Any, Tuple
from tqdm import tqdm 

In [120]:
train_data = defaultdict(list)
test_data = defaultdict(list)

for i in range(5):
    with jsonlines.open(f'train_{i}.jsonl') as f:
        for line in f.iter():
            train_data[i].append(line)

    with jsonlines.open(f'test_{i}.jsonl') as f:
        for line in f.iter():
            test_data[i].append(line)

In [14]:
openai.api_key = 'your_api_key'

In [15]:
# Must modify to your personal fine-tuned model!

ft_model_4 = {
    0: 'ft:gpt-4o-2024-08-06:personal:train0:AcZ1PhB4',
    1: 'ft:gpt-4o-2024-08-06:personal:train1:AcZ2DyL1',
    2: 'ft:gpt-4o-2024-08-06:personal:train2:AcZ3YS9m',
}

ft_model_3 = {
    0: 'ft:gpt-3.5-turbo-1106:personal:train0:AcZ6UY49',
    1: 'ft:gpt-3.5-turbo-1106:personal:train1:AcZ7idHm',
    2: 'ft:gpt-3.5-turbo-1106:personal:train2:AcZ6f95m',
}

In [20]:
time_dict = {
    'days': 24,
    'day': 24,
    'weeks': 24*7,
    'week': 24*7,
    'hours': 1,
    'hour': 1,
    'h': 1,
    'minute': 1/60,
    'min': 1/60,
    's': 1/3600
}

In [123]:
def get_answers(model: str, message: dict, temperature=0.0) -> tuple:
    completion = openai.ChatCompletion.create(
        model=model,
        messages=message['messages'][:2],
        temperature=temperature
    )

    prediction = json.loads(completion.choices[0].message['content'])
    true = json.loads(message['messages'][2]['content'])

    return true, prediction

In [125]:
true_label, prediction_label = defaultdict(list), defaultdict(list)

for i in range(1):
    model = ft_model_3[i]
    testset = test_data[i]

    for j, message in tqdm(enumerate(testset), total=len(testset)):
        try:
            true, prediction = get_answers(model, message)
        except Exception as e:
            print (f'gpt3.5-{i}-{j} : ', e)
            continue
        else:
            true_label[f'gpt3.5-{i}'].append(true)
            prediction_label[f'gpt3.5-{i}'].append(prediction)

2094it [1:01:04,  1.75s/it]


In [126]:
for i in range(1):
    model = ft_model_4[i]
    testset = test_data[i]

    for j, message in tqdm(enumerate(testset), total=len(testset)):
        try:
            true, prediction = get_answers(model, message)
        except Exception as e:
            print (f'gpt4-{i}-{j} : ', e)
            continue
        else:
            true_label[f'gpt4-{i}'].append(true)
            prediction_label[f'gpt4-{i}'].append(prediction)

100%|██████████| 2094/2094 [1:20:43<00:00,  2.31s/it]  


In [128]:
import pickle

with open('true_label_save.pickle', 'wb') as f:
    pickle.dump(true_label, f)

with open('prediction_label_save.pickle', 'wb') as f:
    pickle.dump(prediction_label, f)